In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import cv2
import os

In [2]:
labels_df = pd.read_csv("C:\\python\\archive\\DCSASS Dataset\\Labels\\shoplifting.csv", header=None, names=['filename', 'label', 'value'])

# Data pre-processing

In [3]:
labels_df.head()

,filename,label,value
0,Shoplifting001_x264_0,Shoplifting,0
1,Shoplifting001_x264_1,Shoplifting,0
2,Shoplifting001_x264_2,Shoplifting,0
3,Shoplifting001_x264_3,Shoplifting,0
4,Shoplifting001_x264_4,Shoplifting,0


In [4]:
labels_df['filename'].value_counts().sum()

896

In [5]:
labels_df['value'].value_counts()

value
0    741
1    155
Name: count, dtype: int64

In [6]:
labels_df['label'].value_counts()

label
Shoplifting    896
Name: count, dtype: int64

In [7]:
# Initialize an empty list to store dataframes
import cv2
import os
import pandas as pd
import numpy as np

# Function to preprocess a frame
def preprocess_frame(frame, resize_dim=(224, 224), grayscale=True):
    # Resize the frame
    frame_resized = cv2.resize(frame, resize_dim)
    
    # Convert to grayscale if needed
    if grayscale:
        frame_resized = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    
    return frame_resized

# Path to the main directory containing subfolders with video files
main_dir = "C:\\python\\archive\\DCSASS Dataset\\Shoplifting"

# Load labels from CSV file
labels_df = pd.read_csv("C:\\python\\archive\\DCSASS Dataset\\Labels\\shoplifting.csv", header=None, names=['filename', 'label', 'value'])

dfs = []

# Looping through each subfolder recursively
for root, dirs, files in os.walk(main_dir):
    for dir_name in dirs:
        subdir_path = os.path.join(root, dir_name)
        
        # List files in the subfolder
        video_files = [f for f in os.listdir(subdir_path) if f.endswith('.mp4')]
        
        # Looping through each video file
        for video_file in video_files:
            # Open the video file
            video_path = os.path.join(subdir_path, video_file)
            video_capture = cv2.VideoCapture(video_path)
            
            # Get the label and value for the current video file
            filename = str(video_file)
            filename = filename[:-4]
            label_info = labels_df.loc[labels_df['filename'] == filename]
            if label_info.empty:
                print(f"No label found for {video_file}")
                continue
            
            # Extract the label and value
            label = label_info['label'].iloc[0]
            value = label_info['value'].iloc[0]
            
            # Create a list to store preprocessed frames
            frames = []
            
            # Read and preprocess each frame
            while True:
                ret, frame = video_capture.read()
                if not ret:
                    break
                
                # Preprocess the frame
                processed_frame = preprocess_frame(frame)
                
                # Append the preprocessed frame to the list
                frames.append(processed_frame)
            
            # Combine all frames into a single array
            frames_array = np.array(frames)
            
            # Create a DataFrame for the current video file
            df = pd.DataFrame({'frames': [frames_array], 'label': [label], 'value': [value]})
            
            # Append the DataFrame to the list
            dfs.append(df)
            
            # Release the video capture object
            video_capture.release()
            print(f"Video {video_file} processed successfully.")

# Concatenate all DataFrames in the list
data_df = pd.concat(dfs, ignore_index=True)

# Close all OpenCV windows
cv2.destroyAllWindows()

Video Shoplifting001_x264_0.mp4 processed successfully.
Video Shoplifting001_x264_1.mp4 processed successfully.
Video Shoplifting001_x264_10.mp4 processed successfully.
Video Shoplifting001_x264_11.mp4 processed successfully.
Video Shoplifting001_x264_12.mp4 processed successfully.
Video Shoplifting001_x264_13.mp4 processed successfully.
Video Shoplifting001_x264_14.mp4 processed successfully.
Video Shoplifting001_x264_15.mp4 processed successfully.
Video Shoplifting001_x264_16.mp4 processed successfully.
Video Shoplifting001_x264_17.mp4 processed successfully.
Video Shoplifting001_x264_18.mp4 processed successfully.
Video Shoplifting001_x264_19.mp4 processed successfully.
Video Shoplifting001_x264_2.mp4 processed successfully.
Video Shoplifting001_x264_20.mp4 processed successfully.
Video Shoplifting001_x264_21.mp4 processed successfully.
Video Shoplifting001_x264_22.mp4 processed successfully.
Video Shoplifting001_x264_23.mp4 processed successfully.
Video Shoplifting001_x264_24.mp4 p

In [9]:
data_df['label'].value_counts()

label
Shoplifting    896
Name: count, dtype: int64

In [10]:
data_df.shape

(896, 3)

In [15]:
data_df.head()

,frames,label,value
0,"[[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2...",Shoplifting,0
1,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",Shoplifting,0
2,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",Shoplifting,1
3,"[[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2...",Shoplifting,1
4,"[[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2...",Shoplifting,1


In [17]:
import numpy as np

# Find the maximum length of the largest frame in the dataset
max_frame_length = max(len(frame) for video_frames in data_df['frames'] for frame in video_frames)

# Pad and combine frames for each video
padded_combined_frames = []
for video_frames in data_df['frames']:
    combined_frame = np.concatenate(video_frames, axis=0)
    
    # Check if the combined frame is longer than the maximum frame length
    if len(combined_frame) > max_frame_length:
        # If it's longer, truncate it to the maximum frame length
        padded_combined_frame = combined_frame[:max_frame_length]
    else:
        # Otherwise, pad it with zeros to match the maximum frame length
        padded_combined_frame = np.pad(combined_frame, ((0, max_frame_length - len(combined_frame)), (0, 0)), mode='constant')
    
    padded_combined_frames.append(padded_combined_frame)

# Convert the list of padded combined frames back to a DataFrame column
data_df['padded_combined_frames'] = padded_combined_frames


In [18]:
data_df.drop(columns=['frames','label'],inplace=True)

In [19]:
df.shape

(1, 3)

In [20]:
data_df.head()

,value,padded_combined_frames
0,0,"[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,..."
1,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,1,"[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,..."
4,1,"[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,..."


# Model building and evaluation

In [21]:
import numpy as np
X = np.array(data_df['padded_combined_frames'].tolist())
X_flattened = X.reshape(X.shape[0], -1)
y = np.array(data_df['value'])

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flattened, y, stratify= y, test_size=0.2, random_state=42)

# Define classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000,random_state=42),
    "SVM": SVC(kernel='linear', random_state=42)
}

# Train and evaluate each classifier
for name, classifier in classifiers.items():
    # Train the classifiery
    classifier.fit(X_train, y_train)
    
    # Make predictions
    y_pred = classifier.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    # Print evaluation metrics
    print(f"Classifier: {name}")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    
    # Classification report
    print(classification_report(y_test, y_pred))
    print("\n")


Classifier: Random Forest
Accuracy: 0.8833333333333333
F1 Score: 0.5714285714285714
Precision: 0.7777777777777778
Recall: 0.45161290322580644
              precision    recall  f1-score   support

           0       0.90      0.97      0.93       149
           1       0.78      0.45      0.57        31

    accuracy                           0.88       180
   macro avg       0.84      0.71      0.75       180
weighted avg       0.87      0.88      0.87       180



Classifier: Logistic Regression
Accuracy: 0.8333333333333334
F1 Score: 0.53125
Precision: 0.5151515151515151
Recall: 0.5483870967741935
              precision    recall  f1-score   support

           0       0.90      0.89      0.90       149
           1       0.52      0.55      0.53        31

    accuracy                           0.83       180
   macro avg       0.71      0.72      0.71       180
weighted avg       0.84      0.83      0.84       180



Classifier: SVM
Accuracy: 0.85
F1 Score: 0.5970149253731343
Prec